# Playground for Transformers!

### Attention is all you need 
(https://arxiv.org/abs/1706.03762)

### For software vulnerability detection GYM

This is a minimal example of this **CRAZY** idea!

### Note

* LSTM is working with:
    * Adam Learning rate = 0.01
    * Overfitted on 14 epoch. 100% training accuracy.
    * Worked on 1K dataset sample
    * Using the `hidden`n or `cell` output from LSTM. Not the `output`.
    * Bidrectional (2 layers)
* Transformer:
    * Still trying to find the right combination
    * Adam Learning rate 0.0001
    * Beat this:
     Epoch 46 - Training acc: 0.824250 -Training loss: 0.426513 - Val acc: 0.775586 - Val loss: 0.496733 - Time: 102.7327s
     
* 100K run:  
Epoch 1 - Training acc: 0.674675 -Training loss: 0.601032 - Val acc: 0.700293 - Val loss: 0.565769 - Time: 1541.4087s
Epoch 2 - Training acc: 0.739337 -Training loss: 0.538962 - Val acc: 0.741205 - Val loss: 0.523540 - Time: 1499.5019s
Epoch 3 - Training acc: 0.759912 -Training loss: 0.512259 - Val acc: 0.701826 - Val loss: 0.561835 - Time: 1408.9301s
Epoch 4 - Training acc: 0.762787 -Training loss: 0.509124 - Val acc: 0.758418 - Val loss: 0.498503 - Time: 1412.1519s
Epoch 5 - Training acc: 0.766650 -Training loss: 0.505486 - Val acc: 0.708733 - Val loss: 0.551428 - Time: 1412.9424s
Epoch 6 - Training acc: 0.771212 -Training loss: 0.499414 - Val acc: 0.755797 - Val loss: 0.500520 - Time: 1406.9010s
Epoch 7 - Training acc: 0.773650 -Training loss: 0.496225 - Val acc: 0.766147 - Val loss: 0.481232 - Time: 1409.5581s
Epoch 8 - Training acc: 0.776613 -Training loss: 0.493755 - Val acc: 0.725013 - Val loss: 0.553314 - Time: 1419.0735s
Epoch 9 - Training acc: 0.780300 -Training loss: 0.486875 - Val acc: 0.742471 - Val loss: 0.510461 - Time: 1411.7488s
Epoch 10 - Training acc: 0.785575 -Training loss: 0.480348 - Val acc: 0.782205 - Val loss: 0.456913 - Time: 1414.1186s
Epoch 11 - Training acc: 0.788850 -Training loss: 0.472767 - Val acc: 0.792089 - Val loss: 0.445218 - Time: 1424.2309s
Epoch 12 - Training acc: 0.797475 -Training loss: 0.459784 - Val acc: 0.780095 - Val loss: 0.466281 - Time: 1407.5756s
Epoch 13 - Training acc: 0.804100 -Training loss: 0.448606 - Val acc: 0.780162 - Val loss: 0.482026 - Time: 1438.1905s
Epoch 14 - Training acc: 0.809700 -Training loss: 0.442654 - Val acc: 0.795020 - Val loss: 0.451707 - Time: 1439.5672s
Epoch 15 - Training acc: 0.812675 -Training loss: 0.438664 - Val acc: 0.802461 - Val loss: 0.448924 - Time: 1435.7601s
Epoch 16 - Training acc: 0.819150 -Training loss: 0.428220 - Val acc: 0.793888 - Val loss: 0.473866 - Time: 1419.7033s
Epoch 17 - Training acc: 0.821538 -Training loss: 0.423442 - Val acc: 0.812744 - Val loss: 0.425167 - Time: 1409.6141s
Epoch 18 - Training acc: 0.825037 -Training loss: 0.417003 - Val acc: 0.806414 - Val loss: 0.443101 - Time: 1411.5896s
Epoch 19 - Training acc: 0.827325 -Training loss: 0.411255 - Val acc: 0.790378 - Val loss: 0.482602 - Time: 1413.5015s
Epoch 20 - Training acc: 0.827762 -Training loss: 0.411628 - Val acc: 0.793643 - Val loss: 0.474180 - Time: 1418.7839s
Epoch 21 - Training acc: 0.831237 -Training loss: 0.406693 - Val acc: 0.811878 - Val loss: 0.430180 - Time: 1412.6029s
Epoch 22 - Training acc: 0.832862 -Training loss: 0.405046 - Val acc: 0.796842 - Val loss: 0.491119 - Time: 1411.8020s
Epoch 23 - Training acc: 0.837500 -Training loss: 0.394457 - Val acc: 0.810079 - Val loss: 0.436688 - Time: 1410.8785s
Epoch 24 - Training acc: 0.840300 -Training loss: 0.390056 - Val acc: 0.812278 - Val loss: 0.440547 - Time: 1434.1152s

### Dataset

Import

In [76]:
import numpy as np
import pandas as pd
import os
import torch
import re
import torch.backends.cudnn as cudnn
import torchtext
import matplotlib.pyplot as plt
import time
import torchtext.vocab as vocab
import sklearn.metrics
import torch.nn.functional as F
import random
from torch.autograd import Variable
from torch import nn, optim
from torch.optim import SGD,Adam
from gensim.models.word2vec import Word2Vec
from sklearn.metrics import confusion_matrix

In [77]:
seed = 1234
torch.manual_seed(seed)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
cudnn.benchmark = True
cudnn.enabled = True
print(device)

cpu


Load playset dataset

In [240]:
dataset = pd.read_pickle('playset(0.25.2).pickle')

View dataset

In [241]:
dataset

,functionSource,combine
93792,"go_file_opener_open (GOFileOpener const *fo, g...",False
79770,updatePathMap(bool left_level) {\n\tPoint from...,False
66999,interpret_tilde(const char* path) {\n stati...,False
44284,"checkVarExp(\n Absyn *node,\n Ta...",True
49515,will_have_skip_worktree(const struct cache_ent...,True
...,...,...
96701,"AVLTree_insert(AVLTree * tree, void * data)\n{...",False
67815,"remove_hook(const char *name, hookfn fn)\n{\n\...",False
88363,"output_def(dico_stream_t str, struct gcide_db ...",False
65929,getState(\n\t\tFLMUINT\t\tuiFieldID)\n\t{\n\t\...,False


In [242]:
dataset.describe(include='all')

,functionSource,combine
count,100000,100000
unique,100000,2
top,"GetName(char *str,int strLen) const\n{\n if...",True
freq,1,50000


In [243]:
dataset.functionSource[24492]

'file_replace(struct magic_set *ms, const char *pat, const char *rep)\n{\n\tregex_t rx;\n\tint rc;\n\n\trc = regcomp(&rx, pat, REG_EXTENDED);\n\tif (rc) {\n\t\tchar errmsg[512];\n\t\t(void)regerror(rc, &rx, errmsg, sizeof(errmsg));\n\t\tfile_magerror(ms, "regex error %d, (%s)", rc, errmsg);\n\t\treturn -1;\n\t} else {\n\t\tregmatch_t rm;\n\t\tint nm = 0;\n\t\twhile (regexec(&rx, ms->o.buf, 1, &rm, 0) == 0) {\n\t\t\tms->o.buf[rm.rm_so] = \'\\0\';\n\t\t\tif (file_printf(ms, "%s%s", rep,\n\t\t\t    rm.rm_eo != 0 ? ms->o.buf + rm.rm_eo : "") == -1)\n\t\t\t\treturn -1;\n\t\t\tnm++;\n\t\t}\n\t\tregfree(&rx);\n\t\treturn nm;\n\t}\n}'

### Prepare data (in JSON)

#### Cleaner & Tokkenizer

In [235]:
global c
c=0
def my_tokenizer(code):
    global c
    c+=1
    print('{0}'.format(c), end='\r')
    
    pat = re.compile(r'(/\*([^*]|(\*+[^*/]))*\*+/)|(//.*)')
    code = re.sub(pat,'',code)
    
    ## Remove code comments
    #code = re.sub(r'(/\*([^*]|[\r\n]|(\*+([^*/]|[\r\n])))*\*+/)|(//.*)','',code)
    
    ## Remove newlines & tabs
    code = re.sub('(\n)|(\\\\n)|(\\\\)|(\\t)|(/)|(\\r)','',code)
    
    ## Mix split (characters and words)
    splitter = '\"(.*?)\"| +|(;)|(\()|(==)|(\))|(=)|(\+)|(\-)|(\[)|(\])|(<)|(>)|(\.)|({)'
    code = re.split(splitter,code)
    
    ## Remove None type
    code = list(filter(None, code))
    
    code = list(filter(str.strip, code))
    #code = " ".join(code)
    
    ## Return list of tokens
    return(code)

In [238]:
dataset = dataset.functionSource.apply(my_tokenizer)

In [ ]:
## Change labels boolean to 1 and 0
dataset.iloc[:,1] = np.multiply(dataset.iloc[:,1],1)

In [ ]:
## Change column name
dataset = dataset.rename(columns={'functionSource':'codes', 'combine':'label'})

In [9]:
false = dataset[dataset.iloc[:,1]==0]
true = dataset[dataset.iloc[:,1]==1]

In [10]:
## Split to train,test,valid
train = false[0:20000].append(true[0:1333])
test  = false[20000:22494].append(true[2000:2166])
valid = false[30000:32494].append(true[3000:3166])

## Shuffle
train = train.sample(frac=1).reset_index(drop=True)
test = test.sample(frac=1).reset_index(drop=True)
valid = valid.sample(frac=1).reset_index(drop=True)

In [99]:
## Save to json
train.to_json('.data/train_ratio.json', orient='records',lines=True)
test.to_json('.data/test_ratio.json', orient='records',lines=True)
valid.to_json('.data/valid_ratio.json', orient='records',lines=True)

In [13]:
## Define the field

CODES = torchtext.data.Field(batch_first=True, tokenize=my_tokenizer)
LABEL = torchtext.data.LabelField(dtype=torch.long)
fields = {'codes': ('codes', CODES), 'label': ('label', LABEL)}

In [ ]:
## Import ratio dataset 20K
train_data, valid_data, test_data = torchtext.data.TabularDataset.splits(
                                        path = '.data',
                                        train = 'train_ratio.json',
                                        validation = 'valid_ratio.json',
                                        test = 'test_ratio.json',
                                        format = 'json',
                                        fields = fields)

In [4]:
## Import the 1K data as TabularDataset
train_data, valid_data, test_data = torchtext.data.TabularDataset.splits(
                                        path = '.data',
                                        train = 'train_1k.json',
                                        validation = 'valid_1k.json',
                                        test = 'test_1k.json',
                                        format = 'json',
                                        fields = fields)

In [4]:
## Import the min(10k) data as TabularDataset
train_data, valid_data, test_data = torchtext.data.TabularDataset.splits(
                                        path = '.data',
                                        train = 'train_min.json',
                                        validation = 'valid_min.json',
                                        test = 'test_min.json',
                                        format = 'json',
                                        fields = fields)

In [4]:
## Import the 100K data as TabularDataset
train_data, valid_data, test_data = torchtext.data.TabularDataset.splits(
                                        path = '.data',
                                        train = 'train_100K.json',
                                        validation = 'valid_100k.json',
                                        test = 'test_100k.json',
                                        format = 'json',
                                        fields = fields)

In [6]:
## Testing
print(vars(valid_data[0]))

{'codes': ['ap2sta_data_frame', '(', 'struct', '_adapter', '*adapter,', 'union', 'recv_frame', '*precv_frame,', 'struct', 'sta_info', '**psta', ')', '{', 'u8', '*ptr', '=', 'precv_frame', '-', '>', 'u.hdr.rx_data', ';', 'struct', 'rx_pkt_attrib', '*pattrib', '=', '&precv_frame', '-', '>', 'u.hdr.attrib', ';', 'structsta_priv', '*pstapriv', '=', '&adapter', '-', '>', 'stapriv', ';', 'structmlme_priv', '*pmlmepriv', '=', '&adapter', '-', '>', 'mlmepriv', ';', 'u8', '*mybssid', '=', 'get_bssid', '(', 'pmlmepriv', ')', ';', 'u8', '*myhwaddr', '=', 'myid', '(', '&adapter', '-', '>', 'eeprompriv', ')', ';', 'sint', 'bmcast', '=', 'IS_MCAST', '(', 'pattrib', '-', '>', 'dst', ')', ';', 'if', '(', 'check_fwstate', '(', 'pmlmepriv,', 'WIFI_STATION_STATE', ')', '&&', 'check_fwstate', '(', 'pmlmepriv,', '_FW_LINKED', ')', ')', '{', '*', 'if', 'NULL', '-', 'frame,', 'drop', 'packet', '*if', '(', '(', 'GetFrameSubType', '(', 'ptr', ')', ')', '==', 'WIFI_DATA_NULL', ')', 'return', '_FAIL', ';', '*', 

Doneeeeeeeeeeeeeeeee !

### Vocabulary-related preparation

In [5]:
##### Build the vocabulary

MAX_VOCAB_SIZE = 10000

CODES.build_vocab(train_data, max_size = MAX_VOCAB_SIZE)
LABEL.build_vocab(train_data)

In [6]:
print(f"Unique tokens in TEXT vocabulary: {len(CODES.vocab)}")
print(f"Unique tokens in LABEL vocabulary: {len(LABEL.vocab)}")

Unique tokens in TEXT vocabulary: 10002
Unique tokens in LABEL vocabulary: 2


In [7]:
## Most common word
print(CODES.vocab.freqs.most_common(100))

[(')', 1376013), ('(', 1375892), (';', 1270722), ('-', 656492), ('=', 633211), ('>', 507801), ('{', 307771), ('*', 260258), ('if', 254846), ('0', 200507), ('}', 197609), ('+', 187616), ('[', 166766), (']', 166621), ('return', 131170), ('i', 118283), ('1', 114002), ('==', 106852), ('<', 98924), ('NULL', 87699), ('int', 86141), ('char', 72021), ('the', 69537), ('!', 61198), ('struct', 55464), ('for', 48509), ('else', 45145), ('to', 41012), ('const', 40896), ('&&', 38090), ('#', 32367), ('case', 32198), ('sizeof', 32070), ('break', 30391), ('%s', 30237), ('a', 28443), ('buf', 27300), ('p', 26983), ('len', 25757), ('ret', 25649), ('||', 24762), ('data', 24691), ('2', 24650), ('&', 24588), ('unsigned', 24438), ('is', 23138), ('name', 21712), ('of', 21539), ('this', 20787), ('in', 20509), ('s', 19776), ('n', 19228), ('j', 18797), (':', 18123), ('c', 17438), ('size', 17372), ('file', 16497), ('void', 16480), ('while', 16332), ('%d', 15705), ('we', 14985), ('and', 14598), ('type', 14553), ('no

In [8]:
print(CODES.vocab.itos[:10])
print(LABEL.vocab.stoi)
print(CODES.vocab.stoi[CODES.pad_token])

['<unk>', '<pad>', ')', '(', ';', '-', '=', '>', '{', '*']
defaultdict(None, {0: 0, 1: 1})
1


In [9]:
## place into iterators
train_iterator, valid_iterator, test_iterator = torchtext.data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = 32,
    sort = False)

### Prepare Word2Vec (Optional)

In [ ]:
corpus = pd.read_json('.data/train.json',orient='records',lines=True)

w2v = Word2Vec(corpus.codes, size=300, workers=16, sg=1, min_count=3)
w2v.save('.data/node_w2v_128')

In [10]:
w2v = Word2Vec.load('.data/node_w2v_128')
w2v.wv.vectors

array([[ 0.04268723, -0.01990928, -0.10372822, ...,  0.34928635,
        -0.24622028, -0.02363101],
       [ 0.06655177, -0.08706249, -0.11346684, ...,  0.2967248 ,
        -0.16500187, -0.10260527],
       [ 0.10059763, -0.0993171 , -0.14234892, ...,  0.3913037 ,
        -0.22237949,  0.02339004],
       ...,
       [-0.0031671 ,  0.01939397, -0.00094254, ..., -0.06062187,
        -0.0873417 ,  0.10190531],
       [-0.03719744,  0.02801778,  0.02174594, ..., -0.05577604,
        -0.07265704, -0.00079473],
       [ 0.01092949, -0.03061507, -0.045645  , ..., -0.06999503,
        -0.15679213,  0.11291362]], dtype=float32)

### Transformer class (with LSTM)

In [10]:
class Transformer(nn.Module):
    def __init__(self):
        super(Transformer,self).__init__()
        
        self.embed = nn.Embedding(num_embeddings=10002,
                                  embedding_dim=400)
        self.encode_layer = nn.TransformerEncoderLayer(d_model=400,
                                                       nhead=8,
                                                       dim_feedforward=2048,
                                                       dropout=0.1,
                                                       activation='relu')
        self.trans_encoder = nn.TransformerEncoder(self.encode_layer,
                                                   num_layers=1)
        self.lstm1 = nn.LSTM(input_size=400,
                            hidden_size=128,
                            num_layers=2,
                            batch_first=True,
                            bidirectional=True)
        self.dropout = nn.Dropout(0.3)
        self.fc1 = nn.Linear(128*2,2)
    
    def forward(self,x):
        x = self.embed(x)
        x = self.trans_encoder(x)
        output, (hidden, cell) = self.lstm1(x)
        x = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim=1))
        x = self.fc1(x)
        return(x)

model = Transformer()
model.to(device)
print(model)

Transformer(
  (embed): Embedding(10002, 400)
  (encode_layer): TransformerEncoderLayer(
    (self_attn): MultiheadAttention(
      (out_proj): Linear(in_features=400, out_features=400, bias=True)
    )
    (linear1): Linear(in_features=400, out_features=2048, bias=True)
    (dropout): Dropout(p=0.1, inplace=False)
    (linear2): Linear(in_features=2048, out_features=400, bias=True)
    (norm1): LayerNorm((400,), eps=1e-05, elementwise_affine=True)
    (norm2): LayerNorm((400,), eps=1e-05, elementwise_affine=True)
    (dropout1): Dropout(p=0.1, inplace=False)
    (dropout2): Dropout(p=0.1, inplace=False)
  )
  (trans_encoder): TransformerEncoder(
    (layers): ModuleList(
      (0): TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): Linear(in_features=400, out_features=400, bias=True)
        )
        (linear1): Linear(in_features=400, out_features=2048, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_featur

### Load Word2Vec weights to embeddings (Optional)

In [14]:
weights = torch.FloatTensor(w2v.wv.vectors)
weights = weights.to(device)
model.embed = model.embed.from_pretrained(weights)
#model.embed = model.embed.weight.data.copy_(weights)

### Parameters

In [11]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 9,507,394 trainable parameters


### Prepare accuracy function

In [12]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(preds)
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [13]:
def softmax_accuracy(probs,all_labels):
    def getClass(x):
        return(x.index(max(x)))
    
    all_labels = all_labels.tolist()
    probs = pd.Series(probs.tolist())
    all_predicted = probs.apply(getClass)
    all_predicted.reset_index(drop=True, inplace=True)
    vc = pd.value_counts(all_predicted == all_labels)
    try:
        acc = vc[1]/len(all_labels)
    except:
        if(vc.index[0]==False):
            acc = 0
        else:
            acc = 1
    return(acc)

### Loss functions

In [14]:
## Define optimizer
#optimizer = SGD(model.parameters(), lr = 0.01)
optimizer = Adam(model.parameters(), lr=0.0001, weight_decay=0.01)

## Define loss function
#criterion = nn.BCELoss().to(device) ## Sigmoid activation function
#criterion = nn.NLLLoss().to(device) ### Log_softmax activation
criterion = nn.CrossEntropyLoss().to(device) ## No activation function bcs softmax included

### Training

In [15]:
epochs=100
for e in range(epochs):
    running_acc = 0
    running_loss = 0
    timer = time.time()
    model.train()

    for batch in train_iterator:
        batch.codes, batch.label = batch.codes.to(device), batch.label.to(device)
        optimizer.zero_grad()
        output = model(batch.codes)
        loss = criterion(output, batch.label)
        loss.backward()
        optimizer.step()
        acc = softmax_accuracy(output,batch.label)
        running_acc += acc.item()
        running_loss += loss.item()
    else:
        with torch.no_grad():
            model.eval()
            running_acc_val = 0
            running_loss_val = 0
            for batch in valid_iterator:
                batch.codes, batch.label = batch.codes.to(device), batch.label.to(device)
                output_val = model(batch.codes)
                loss_val = criterion(output_val,batch.label)
                acc_val = softmax_accuracy(output_val,batch.label)
                running_acc_val += acc_val.item()
                running_loss_val += loss_val.item()
        
        print("Epoch {} - Training acc: {:.6f} -Training loss: {:.6f} - Val acc: {:.6f} - Val loss: {:.6f} - Time: {:.4f}s".format(e+1, running_acc/len(train_iterator), running_loss/len(train_iterator), running_acc_val/len(valid_iterator), running_loss_val/len(valid_iterator), (time.time()-timer)))

Epoch 1 - Training acc: 0.674675 -Training loss: 0.601032 - Val acc: 0.700293 - Val loss: 0.565769 - Time: 1541.4087s
Epoch 2 - Training acc: 0.739337 -Training loss: 0.538962 - Val acc: 0.741205 - Val loss: 0.523540 - Time: 1499.5019s
Epoch 3 - Training acc: 0.759912 -Training loss: 0.512259 - Val acc: 0.701826 - Val loss: 0.561835 - Time: 1408.9301s
Epoch 4 - Training acc: 0.762787 -Training loss: 0.509124 - Val acc: 0.758418 - Val loss: 0.498503 - Time: 1412.1519s
Epoch 5 - Training acc: 0.766650 -Training loss: 0.505486 - Val acc: 0.708733 - Val loss: 0.551428 - Time: 1412.9424s
Epoch 6 - Training acc: 0.771212 -Training loss: 0.499414 - Val acc: 0.755797 - Val loss: 0.500520 - Time: 1406.9010s
Epoch 7 - Training acc: 0.773650 -Training loss: 0.496225 - Val acc: 0.766147 - Val loss: 0.481232 - Time: 1409.5581s
Epoch 8 - Training acc: 0.776613 -Training loss: 0.493755 - Val acc: 0.725013 - Val loss: 0.553314 - Time: 1419.0735s
Epoch 9 - Training acc: 0.780300 -Training loss: 0.48687

KeyboardInterrupt: 

In [16]:
import sklearn.metrics
from sklearn.metrics import confusion_matrix

model.eval()
with torch.no_grad():
    running_acc_test = 0
    running_loss_test = 0
    all_pred=[]
    all_labels=[]
    for batch in train_iterator:
        batch.codes, batch.label = batch.codes.to(device), batch.label.to(device)
        output_test = model(batch.codes).squeeze(1)
        loss_test = criterion(output_test,batch.label)
        acc_test = softmax_accuracy(output_test,batch.label)
        running_acc_test += acc_test.item()
        running_loss_test += loss_test.item()
        all_pred += output_test.tolist()
        all_labels += batch.label.tolist()


print('Train acc: ',running_acc_test/len(train_iterator))
print('Train loss: ',running_loss_test/len(train_iterator))


def getClass(x):
    return(x.index(max(x)))

probs = pd.Series(all_pred)
all_predicted = probs.apply(getClass)
all_predicted.reset_index(drop=True, inplace=True)
vc = pd.value_counts(all_predicted == all_labels)

confusion = sklearn.metrics.confusion_matrix(y_true=all_labels, y_pred=all_predicted)
print('Confusion matrix: \n',confusion)

Train acc:  0.852
Train loss:  0.3846263731718063
Confusion matrix: 
 [[3562  438]
 [ 746 3254]]


### Testing

In [17]:
### SOFTMAX


model.eval()
with torch.no_grad():
    running_acc_test = 0
    running_loss_test = 0
    all_pred=[]
    all_labels=[]
    for batch in test_iterator:
        batch.codes, batch.label = batch.codes.to(device), batch.label.to(device)
        output_test = model(batch.codes).squeeze(1)
        loss_test = criterion(output_test,batch.label)
        acc_test = softmax_accuracy(output_test,batch.label)
        running_acc_test += acc_test.item()
        running_loss_test += loss_test.item()
        all_pred += output_test.tolist()
        all_labels += batch.label.tolist()


print('Test acc: ',running_acc_test/len(test_iterator))
print('Test loss: ',running_loss_test/len(test_iterator))


def getClass(x):
    return(x.index(max(x)))

probs = pd.Series(all_pred)
all_predicted = probs.apply(getClass)
all_predicted.reset_index(drop=True, inplace=True)
vc = pd.value_counts(all_predicted == all_labels)

probs2=[]
for x in probs:
    probs2.append(x[1])

confusion = sklearn.metrics.confusion_matrix(y_true=all_labels, y_pred=all_predicted)
print('Confusion matrix: \n',confusion)


tn, fp, fn, tp = confusion.ravel()
print('\nTP:',tp)
print('FP:',fp)
print('TN:',tn)
print('FN:',fn)

## Performance measure
print('\nAccuracy: '+ str(sklearn.metrics.accuracy_score(y_true=all_labels, y_pred=all_predicted)))
print('Precision: '+ str(sklearn.metrics.precision_score(y_true=all_labels, y_pred=all_predicted)))
print('Recall: '+ str(sklearn.metrics.recall_score(y_true=all_labels, y_pred=all_predicted)))
print('F-measure: '+ str(sklearn.metrics.f1_score(y_true=all_labels, y_pred=all_predicted)))
print('Precision-Recall AUC: '+ str(sklearn.metrics.average_precision_score(y_true=all_labels, y_score=probs2)))
print('AUC: '+ str(sklearn.metrics.roc_auc_score(y_true=all_labels, y_score=probs2)))
print('MCC: '+ str(sklearn.metrics.matthews_corrcoef(y_true=all_labels, y_pred=all_predicted)))

Test acc:  0.8237819488817891
Test loss:  0.434028828153595
Confusion matrix: 
 [[4235  765]
 [ 999 4001]]

TP: 4001
FP: 765
TN: 4235
FN: 999

Accuracy: 0.8236
Precision: 0.8394880402853546
Recall: 0.8002
F-measure: 0.8193733360638952
Precision-Recall AUC: 0.8626872364818003
AUC: 0.8773458799999999
MCC: 0.6479099280617139


In [ ]:
#### BINARY

import sklearn.metrics
from sklearn.metrics import confusion_matrix

with torch.no_grad():
    running_acc_test = 0
    running_loss_test = 0
    all_pred=[]
    all_labels=[]
    for batch in test_iterator:
        batch.codes = batch.codes.to(device)
        batch.label = batch.label.to(device)
        output_test = model(batch.codes).squeeze(1)
        loss_test = criterion(output_test,batch.label)
        acc_test = binary_accuracy(output_test,batch.label)
        running_acc_test += acc_test.item()
        running_loss_test += loss_test.item()
        all_pred += torch.round(output_test).tolist()
        all_labels += batch.label.tolist()


print('Test acc: ',running_acc_test/len(test_iterator))
print('Test loss: ',running_loss_test/len(test_iterator))

confusion = sklearn.metrics.confusion_matrix(y_true=all_labels, y_pred=all_pred)
print('Confusion matrix: \n',confusion)

# RESULTS